# Session 8: Abstract Classes, Interfaces, and Error Handling

We've talked about the `protected` keyword and seen some interfaces in previous sessions when dealing with `IEnumerable` but haven't gotten into the science of Object Oriented Programming (OOP) and how we can create a hierarchy of objects that we can work with in our applications.  In some of the .NET frameworks like ASP.NET Core, interfaces and abstract classes are used in many places and are key to your successful use of the framework.

## Abstract Classes

Everything in C# is an object.  A class in C# can derive from another class in a **is a** relationship.  Let's look at the common Shape sample:

In [ ]:
abstract class Shape {
    
}

class Triangle : Shape { }
class Rectangle : Shape { }
class Square : Rectangle { }

In C#, we declare a class is abstract with the `abstract` keyword.  This prevents anyone from being able to create this type of class directly and they can only `inherit` from it using the `: ClassName` notation you see in `Triangle`, `Rectangle`, and `Square`.  `Square` inherits from Rectangle so a `Square` **is a** `Rectangle` and also a `Shape`

We can add methods to the `Shape` and they will be available in all of the other classes: 

In [ ]:
abstract class Shape {
    public int Perimeter() { return 10; }
    public decimal Area() { return 15; }
}

class Triangle : Shape { }
class Rectangle : Shape { }
class Square : Rectangle { }

// consequently, we can execute the perimeter and area methods 
// on Triangle, Rectangle, and Square
var t = new Square();
display(t.Perimeter());

However, we know that `Perimeter` and `Area` are calculated differently for each shape.  We can add properties specific to each class, declare the `Perimiter` and `Area` methods as `abstract` and `override` them in each of the derived shape classes.  An `abstract method` or `abstract property`:
- **Only appears** in an abstract class
- **MUST** be implemented in a class that inherits from it. 
- Provides no body definition in the abstract class

In [ ]:
abstract class Shape {
    public abstract int Perimeter();
    public abstract decimal Area();
}

class Triangle : Shape { 
    public int Side1Length;
    public int Side2Length;
    public int Side3Length;
    
    public override int Perimeter() {
        return Side1Length + Side2Length + Side3Length;
    }
    
    public override decimal Area() {
        var semiPerimeter = 0.5 * Perimeter();
        
        // Use Heron's Formula
        return (decimal)Math.Sqrt(semiPerimeter * (semiPerimeter - Side1Length) * (semiPerimeter - Side2Length) * (semiPerimeter - Side3Length));
    }
    
}
class Rectangle : Shape {

    public int Length;
    public int Width;
    
    public override int Perimeter() {
        return Length*2 + Width*2;
    }
    
    public override decimal Area() {
        return Length * Width;
    }
    

}
class Square : Rectangle { }

// consequently, we can execute the perimeter and area methods on Triangle, Rectangle, and Square
var t = new Triangle() { Side1Length=3, Side2Length=4, Side3Length=5};
display(t.Area());
display(t.Perimeter());

/*
var rect = new Rectangle { Length=3, Width=4 };
display(rect.Perimeter());
*/

6

12

Side1Length,Side2Length,Side3Length
3,4,5


You can declare variables or parameters of the base type and pass in or assign the derived type.  This gives your classes more flexibility in their interactions.

In [ ]:
abstract class Shape {
    public int Perimeter() { return 10; }
    public decimal Area() { return 15; }
}

class Triangle : Shape { }
class Rectangle : Shape { }
class Square : Rectangle { }

Shape s = new Rectangle();
display(s.Area());

bool IsQuadrilateral(Shape s) {
    return (s is Rectangle);
}

Shape t = new Triangle();
display("Rectangle is quadrilateral: " + IsQuadrilateral(s));
display("Triangle is quadrilateral: " + IsQuadrilateral(t));

15

Rectangle is quadrilateral: True

Triangle is quadrilateral: False

### Virtual Methods

There are times when you want to provide a default implementation of a method, but provide the ability for subclasses to be able to override or add value to that method.  This is done through the use of the `virtual` keyword in the base class and the child classes use the `override` keyword or `new` keyword to re-implement those methods.  [New](https://docs.microsoft.com/en-us/dotnet/csharp/language-reference/keywords/new-modifier) will hide the base implementation and prevent any compiler errors.

In [ ]:
abstract class Shape {
    public abstract int Perimeter();
    public virtual decimal Area() {
        return Length * Width;
    }

    public int Length;
    public int Width;
    
}

class Triangle : Shape { 
    public int Side1Length;
    public int Side2Length;
    public int Side3Length;
    
    public override int Perimeter() {
        return Side1Length + Side2Length + Side3Length;
    }
    
    public override decimal Area() {
        var semiPerimeter = 0.5 * Perimeter();
        
        // Use Heron's Formula
        return (decimal)Math.Sqrt(semiPerimeter * (semiPerimeter - Side1Length) * (semiPerimeter - Side2Length) * (semiPerimeter - Side3Length));
    }
    
}
class Rectangle : Shape {

    public override int Perimeter() {
        return Length*2 + Width*2;
    }
    
}
class Square : Rectangle { }

// consequently, we can execute the perimeter and area methods on Triangle, Rectangle, and Square
var t = new Triangle() { Side1Length=3, Side2Length=4, Side3Length=5};
display(t.Area());
display(t.Perimeter());
display(t);

6

12

Side1Length,Side2Length,Side3Length,Length,Width
3,4,5,0,0


### [Sealed Classes](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/classes-and-structs/abstract-and-sealed-classes-and-class-members?WT.mc_id=visualstudio-twitch-jefritz)

You can prevent another class from inheriting a class by using the `sealed` keyword.  Members of the class that were previously marked virtual are no longer available to be overridden.

In [ ]:
abstract class Shape {
    public abstract int Perimeter();
    public abstract decimal Area();

    public int Length;
    public int Width;

}

// Try marking this class as sealed
class Rectangle : Shape {

    public override int Perimeter() {
        return Length*2 + Width*2;
    }
    
    // Also try marking this method as sealed
    public override decimal Area() {
        return Length * Width;
    }
    
}
class Square : Rectangle { 

    public override decimal Area() {
        return Length*Length;
    }

}

var s = new Square() { Length=4, Width=4};
display(s.Area());

hello

16

## [Interfaces](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/interfaces?WT.mc_id=visualstudio-twitch-jefritz)

Interfaces define behaviors that a class or a struct **MUST** implement.  An interface can contain methods, properties, events and indexers.  They can also contain static methods with an implementation.  We previously saw collection objects that implement the `IEnumerable<T>` interface that allows LINQ methods to be executed against them.  These members in the interface are declared without scope, as they are required to be accessible to any object that wishes to interact with it.

Assign one or multiple interfaces to a class or struct using the same `:` notation, separating each interface name with a comma.  You must then implement the content of the interface in your object's definition.

By convention, many C# developers name their interfaces starting with the letter I. 

In [ ]:
interface ILogger {

    void LogError(string error);
    
    int ErrorCount { get; }
    
}

class DisplayLogger : ILogger {
    
    private static int _ErrorCount = 0;
    
    public void LogError(string error) {
        display("There was an error: " + error);
        _ErrorCount++;
    }
    
    public int ErrorCount { get { return _ErrorCount; }  }
    
}

class DbLogger : ILogger {

    public void LogError(string error) {
        // do something in the database to log the error
        display("Error logged to database");
    }
    
    public int ErrorCount { 
        get {
            // query the database for the count of errors
            return 0;
        }
    }
    
}

long DisplayErrorCount(ILogger logger) {
    // do something with the eventlog object
    return logger.ErrorCount;
}

ILogger logger = new DisplayLogger();
logger.LogError("Houston, we have a problem...");
display(DisplayErrorCount(logger));

There was an error: Houston, we have a problem...

1

### Explicit Implementation

Implementing methods of an interface without any modifiers in the previous example makes them visible to any interactions with the class type.  It is also possible to HIDE the interface so that it is only visible when the class is explicitly cast to the interface type.

In [ ]:
public interface IDoAThing {
    void DoTheThing();
}

class PublicThing : IDoAThing {
    public void DoTheThing() {
        display("I did the thing!");
    }
}

class SecretThing : IDoAThing {
    void IDoAThing.DoTheThing() {
        display("Shh...  I did a thing");
    }
}

// let's try to DoTheThing
IDoAThing p = new SecretThing();
p.DoTheThing();

Shh...  I did a thing

### Default Implementations

Interfaces can implement other interfaces and can also provide default implementations of their methods.  A classic example of this use is to create new versions of interfaces without breaking compatibility with older interfaces.

In [ ]:
public interface IDoAThing2 : IDoAThing {
    void DoSomethingElse();
    void IDoAThing.DoTheThing() {
        display("This is my default thing");
    }
}

class NewThing : IDoAThing2 {
    public void DoSomethingElse() {
        display("Something else?");
    }

		public void DoTheThing() 
		{
			display("I did the thing!");
		}

}

// let's try to DoTheThing
IDoAThing2 p = new NewThing();
p.DoSomethingElse();
p.DoTheThing();

Something else?

I did the thing!

## [Error Handling + Exceptions](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/exceptions?WT.mc_id=visualstudio-twitch-jefritz)

In C# and .NET, we work with errors through the `System.Exception` object.  This object can carry information about the site of the error, the type of the error, and includes a stacktrace to assist in debugging.

In [ ]:
decimal Divide(decimal arg1, decimal arg2) {
    return arg1 / arg2;
}

display(Divide(4, 2));

2

### try...catch

For methods that we want to prevent interactions that could throw errors, we can wrap those commands in a `try..catch` block.  The `catch` block can receive an `Exception` type to allow for inspection,  Similarly to `if..else if...else` blocks, you can have multiple `catch` blocks to catch different exceptions with a final default that does not include an Exception argument to continue processing appropriately.

If you would like to add information about the error, you can `throw` the exception to notify other calling methods about the error type incurred.

In [ ]:
decimal Divide(decimal arg1, decimal? arg2) {

    try {
        return arg1 / arg2.Value;
    } catch (DivideByZeroException e) {
        display("Division by zero is not allowed");
        return 0;
    } catch {
        display("You broke something else");
        throw;
    }
}

display(Divide(4, 2));

You broke something else

Error: System.InvalidOperationException: Nullable object must have a value.
   at System.Nullable`1.get_Value()
   at Submission#53.Divide(Decimal arg1, Nullable`1 arg2)
   at Submission#53.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### [Clean up after try...catch using finally](https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/exceptions/how-to-execute-cleanup-code-using-finally?WT.mc_id=visualstudio-twitch-jefritz)

A good practice is to clean up the state of your methods that threw an exception, and we use the `finally` block to execute code after a `catch` block has been executed.  The `finally` block is **ALWAYS** executed when your application throws an error.  This is particularly effective when you are working with external resources like a file on disk or a database connection. 

In [ ]:
decimal DivideCleanup(decimal arg1, decimal? arg2) {

    try {
        return arg1 / arg2.Value;
    } catch {
        display("You broke something else");
        throw;
    } finally {
        display("Cleaning up code here...");
    }
}

display(DivideCleanup(4, 2));

Cleaning up code here...

2

### Create our own Exception

As the complexity of your applications grows, you should create your own exception objects that more clearly define the error scenario that has been reached.  This allows your developer teammates and customers to receive richer error reports and triage those problems appropriately.  You can inherit from the base Exception object.  By convention, C# and .NET developers typically name their exception classes **Something**Exception to make it clear that the object contains error information.  

There are four constructor signatures that you _SHOULD_ implement in your custom exception class, and then you can implement any other properties or methods you would like.  These are not required, but are considered good practice:

In [ ]:
public class BetterDivisionException : System.Exception
{
    public BetterDivisionException() : base("Unable to divide by zero") { }
    public BetterDivisionException(string message) : base(message) { }
    public BetterDivisionException(string message, System.Exception inner) : base(message, inner) { }

    // A constructor is needed for serialization when an
    // exception propagates from a remoting server to the client.
    protected BetterDivisionException(System.Runtime.Serialization.SerializationInfo info,
        System.Runtime.Serialization.StreamingContext context) : base(info, context) { }
    
    // Remember, division is Dividend / Divisor
    
    public decimal? Dividend { get; set; }
    public decimal? Divisor { get; set; }
    
}

decimal DivideHandled(decimal arg1, decimal? arg2) {

/**/
    if (arg2 == null) throw new BetterDivisionException("Cannot divide by null")    { Dividend=arg1, Divisor=arg2};
    if (arg2.Value == 0) throw new BetterDivisionException("Cannot divide by zero") { Dividend=arg1, Divisor=arg2};
/**/

    return arg1 / arg2.Value;

}

try {

    display(DivideHandled(4, 0));

} catch (BetterDivisionException e) {
    display(e.Message);
    display($"{e.Dividend} / {e.Divisor}");
    display(e.StackTrace);
} catch (DivideByZeroException e) {
    display("Dividing by zero");
    throw;
} catch {
    display("Did something else");
    throw;
}

Cannot divide by zero

4 / 0

   at Submission#58.DivideHandled(Decimal arg1, Nullable`1 arg2)
   at Submission#58.<<Initialize>>d__0.MoveNext()